#### Загрузка словаря

In [1]:
! wget https://opencorpora.org/files/export/dict/dict.opcorpora.xml.zip
! unzip dict.opcorpora.xml.zip

--2025-09-23 09:54:56--  https://opencorpora.org/files/export/dict/dict.opcorpora.xml.zip
Resolving opencorpora.org (opencorpora.org)... 104.21.15.199, 172.67.163.210, 2606:4700:3030::6815:fc7, ...
Connecting to opencorpora.org (opencorpora.org)|104.21.15.199|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28757314 (27M) [application/zip]
Saving to: ‘dict.opcorpora.xml.zip.1’

dict.opcorpora.xml. 100%[===================>]  27.42M  10.3MB/s    in 2.7s    

2025-09-23 09:54:59 (10.3 MB/s) - ‘dict.opcorpora.xml.zip.1’ saved [28757314/28757314]

Archive:  dict.opcorpora.xml.zip
replace dict.opcorpora.xml? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


#### Считывание исходного словаря и создание словаря для лемматизатора

In [2]:
from lxml import etree

def creat_dict():
    dict_ = {}
    lemmas = etree.iterparse('dict.opcorpora.xml', events=("end",), tag="lemma", recover=True)

    for _ , lemma in lemmas:
        l = lemma.find("l")

        lemma_text = l.attrib.get("t")
        g = l.findall("g")
        tag = g[0].attrib.get("v", "Unknown")

        for f in lemma.findall("f"):
            word = f.attrib.get("t")
            dict_[word] = [lemma_text, tag]

        lemma.clear()
    return dict_

dict_ = creat_dict()

#### Проверка работы словаря

In [15]:
print(dict_['стала'])

['стал', 'VERB']


#### Считывание текста

In [12]:
with open('text.txt', 'r', encoding='utf-8') as f:
    text = f.read()

#### Преобразование текста к формату для обработки

In [13]:
import string

def to_form(text):
  clean_text = "".join(ch for ch in text if ch not in string.punctuation)
  words = clean_text.strip().split()
  print(words)

  return words

words = to_form(text)

['Стала', 'стабильнее', 'экономическая', 'и', 'политическая', 'обстановка', 'предприятия', 'вывели', 'из', 'тени', 'зарплаты', 'сотрудников', 'Все', 'Гришины', 'одноклассники', 'уже', 'побывали', 'за', 'границей', 'он', 'был', 'чуть', 'ли', 'не', 'единственным', 'кого', 'не', 'вывозили', 'никуда', 'дальше', 'Красной', 'Пахры', 'Привет', 'привет']


#### Лемматизация

In [17]:
result = []
for w in words:
  if w.lower() not in dict_.keys():
    result.append(f"{w}{{Unknown}}")
  else:
    lemma, tag = dict_[w.lower()]
    result.append(f"{w}{{{lemma}={tag}}}")

text_res = " ".join(result)
print(text_res)


Стала{стал=VERB} стабильнее{стабильнее=COMP} экономическая{экономический=ADJF} и{и=NOUN} политическая{политический=ADJF} обстановка{обстановка=NOUN} предприятия{предприятие=NOUN} вывели{вывел=VERB} из{иза=NOUN} тени{тень=NOUN} зарплаты{зарплата=NOUN} сотрудников{сотрудник=NOUN} Все{весь=ADJF} Гришины{гришин=ADJF} одноклассники{одноклассник=NOUN} уже{уже=PRCL} побывали{побывал=VERB} за{за=PREP} границей{граница=NOUN} он{он=NPRO} был{есть=VERB} чуть{чуть=CONJ} ли{ли=NOUN} не{не=PRCL} единственным{единственный=ADJF} кого{кто=NPRO} не{не=PRCL} вывозили{вывозил=VERB} никуда{никуда=ADVB} дальше{дальше=COMP} Красной{красный=ADJF} Пахры{пахра=NOUN} Привет{привет=INTJ} привет{привет=INTJ}


#### Сохранение в файл

In [18]:
with open("output.txt", "w", encoding="utf-8") as f:
    f.write(text_res)
